In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from os import listdir
import shutil
import zipfile
import tensorflow as tf

from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

train_zip='/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'
zip_ref=zipfile.ZipFile(train_zip,'r')
zip_ref.extractall('/kaggle/working/')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

work='/kaggle/working'
train_path='/kaggle/working/train/'
file_items=os.listdir(train_path)

fig,([ax1,ax2,ax3,ax4,ax5],[ax6,ax7,ax8,ax9,ax10])=plt.subplots(2,5,figsize=(15,10))
figs=[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10]

for i in range(10):
    k=mpimg.imread(os.path.join(train_path,file_items[i]))
    figs[i].imshow(k)

In [ ]:
!mkdir '/kaggle/working/train/dog'
!mkdir '/kaggle/working/train/cat'

In [ ]:

for f in listdir(train_path):
    if os.path.isfile(os.path.join(train_path, f)):
        if "dog" in f:
            shutil.move(os.path.join(train_path, f), os.path.join('/kaggle/working/train/dog', f))
        if "cat" in f:
            shutil.move(os.path.join(train_path, f), os.path.join('/kaggle/working/train/cat', f))

In [ ]:
dog_train_path='/kaggle/working/train/dog'
cat_train_path='/kaggle/working/train/cat'

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))

In [ ]:

train_dir = "/kaggle/working/train"
BATCH_SIZE = 8

train_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
      preprocessing_function= tf.keras.applications.resnet50.preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True,shear_range=0.2,
    zoom_range=0.1,
    validation_split=0.05
    )

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(200, 200), 
                                                    batch_size=BATCH_SIZE,
    subset='training', class_mode='binary'
                                                   )

validation_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(200, 200), 
                                                    batch_size=BATCH_SIZE, subset='validation', class_mode='binary')

In [ ]:

train_zip='/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip'
zip_ref=zipfile.ZipFile(train_zip,'r')
zip_ref.extractall('/kaggle/test/')
test_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
      preprocessing_function= tf.keras.applications.resnet50.preprocess_input
    )

test_datagen = test_datagen.flow_from_directory('/kaggle/test/', 
                                                    target_size=(200, 200), 
                                                    batch_size=32,shuffle = False)


Training the Resnet model with one FC hidden layer

In [ ]:
model = Sequential()
model.add(tf.keras.applications.resnet50.ResNet50(include_top = False, pooling = 'max', weights = 'imagenet'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.layers[0].trainable = False 
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'binary_crossentropy')


In [ ]:
model.fit_generator(train_generator, epochs = 5, validation_data = validation_generator)

In [ ]:
history = model.history
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(test_datagen)

Submitting the below file which gave loss score of 0.075

In [ ]:
files = [f.split('.')[0][5:] for f in test_datagen.filenames]
solution = pd.DataFrame({"id": files, "label":pred[:,0]})
solution.to_csv("dogsVScatsRes3.csv", index = False)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


Ensemble model with VGG19 InceptionV3 and resnet

In [ ]:
vgg = tf.keras.applications.vgg19.VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3),pooling="max")

for layers in vgg.layers:
    layers.trainable=False

print(vgg.output)

In [ ]:
inc = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3),pooling="max")

for layers in inc.layers:
    layers.trainable=False

print(inc.output)

In [ ]:
feature_list = []
y=[]
for path in listdir(dog_train_path):
    x = load_img(dog_train_path+'/'+path,target_size=(224,224))
    img_array = img_to_array(x)
    img_array = np.expand_dims(img_array, axis=0)
    features = vgg.predict(img_array)
    feature_list.append(features)
    
feat_lst = np.reshape(feature_list,(-1,7*7*512))
y=[1]*feat_lst.shape[0]
feature_list = []
for path in listdir(cat_train_path):
    x = load_img(cat_train_path+'/'+path,target_size=(224,224))
    img_array = img_to_array(x)
    img_array = np.expand_dims(img_array, axis=0)
    features = vgg.predict(img_array)
    feature_list.append(features)
    
feat_lst = np.append(feat_lst ,np.reshape(feature_list,(-1,7*7*512)), axis=0)
y=y+[0]*feat_lst.shape[0]

In [ ]:
fs= [dog_train_path+'/'+path for path in listdir(dog_train_path)] + [cat_train_path+'/'+path for path in listdir(cat_train_path)]

In [ ]:
resent = tf.keras.applications.resnet50.ResNet50(include_top = False, pooling = 'max', weights = 'imagenet')

In [ ]:
y=[]
feature_list = []
for path in fs:
    x = load_img(path,target_size=(224,224))
    img_array = img_to_array(x)
    img_array = np.expand_dims(img_array, axis=0)
    feature_list.append(np.concatenate([resent.predict(img_array)[0],inc.predict(img_array)[0],vgg.predict(img_array)[0]]))
    y.append(1 if 'dog' in path else 0)

In [ ]:
import pickle
input_file = open("resent.pkl",'wb')
pickle.dump({"x":feature_list,'y':y},input_file)

In [ ]:
import pickle
input_file = open("/kaggle/input/resenet1/resent (1).pkl",'rb')
db = pickle.load(input_file)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(db['x'], db['y'], test_size=0.10, random_state=42)

Training a lighgbm model 

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred ))
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
pred = []
files=[]
testpath = '/kaggle/test/test'
for path in listdir(testpath):
    files.append(path)
    x = load_img(testpath+'/'+path,target_size=(224,224))
    img_array = img_to_array(x)
    img_array = np.expand_dims(img_array, axis=0)
    pred.append(clf.predict_proba([np.concatenate([resent.predict(img_array)[0],inc.predict(img_array)[0],vgg.predict(img_array)[0]])]))

The below file gave loss of 0.078

In [ ]:
files = [f.split('.')[0] for f in files]
solution2 = pd.DataFrame({"id": files, "label":np.array(pred).reshape(12500,2)[:,1]})
solution2.to_csv("dogsVScats.csv", index = False)

FInal submission is avg of both scores which gave loss of 0.068

References:
https://www.kaggle.com/code/sanchitvj/cat-or-dog-transfer-learning-using-resnets
https://towardsdatascience.com/transfer-learning-for-image-classification-using-keras-c47ccf09c8c8